# SEC EDGAR PARSER

### Description
blah blah blah ( I'll fill it out later )

### All imports

In [1]:
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup
import pandas as pd
import requests
import shutil
import os

### Get Tickers and CIK-keys

#### If you haven't tickers_and_cik.txt

In [2]:
data = pd.read_excel('Screener tests.xlsx')
top_data = data[data['Cap M$'] != 'No data on finviz'][:100]
tickers = list(top_data['Все'])

In [3]:
cik_dic = {}

for ticker in tickers:
    
    url = f'https://sec.report/Ticker/{ticker}/'
    
    request = requests.get(url, headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36'
      })
    content = request.content
    
    try:
        soup = BeautifulSoup(content, 'html')
        cik = soup.find('h2').text
        cik_dic[ticker] = cik.split(' ')[2]
    except:
        pass

In [5]:
len(cik_dic)

98

#### If you have tickers_and_cik.txt

In [2]:
cik_dic = {}

with open('tickers_and_cik.txt', 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        cik_dic[line.split(' ')[0]] = line.split(' ')[1]

In [3]:
cik_dic

{'AAPL': '0000320193',
 'MSFT': '0000789019',
 'AMZN': '0001018724',
 'GOOG': '0001652044',
 'GOOGL': '0001652044',
 'TSLA': '0001318605',
 'FB': '0001326801',
 'BABA': '0001577552',
 'BRK-B': '0001067983',
 'BRK-A': '0001067983',
 'TSM': '0001046179',
 'V': '0001403161',
 'JPM': '0000019617',
 'JNJ': '0000200406',
 'WMT': '0000104169',
 'PG': '0000080424',
 'UNH': '0000731766',
 'NVDA': '0001045810',
 'MA': '0001141391',
 'DIS': '0001744489',
 'BAC': '0000070858',
 'HD': '0000354950',
 'PYPL': '0001633917',
 'TM': '0001094517',
 'INTC': '0000050863',
 'VZ': '0000732712',
 'NVS': '0001114448',
 'ASML': '0000937966',
 'CMCSA': '0001166691',
 'NKE': '0000320187',
 'ADBE': '0000796343',
 'NFLX': '0001065280',
 'XOM': '0000034088',
 'KO': '0000021344',
 'T': '0000732717',
 'MRK': '0000310158',
 'PDD': '0001737806',
 'PFE': '0000078003',
 'CRM': '0001108524',
 'ABBV': '0001551152',
 'TMO': '0000097745',
 'PEP': '0000077476',
 'ABT': '0000001800',
 'CSCO': '0000858877',
 'AVGO': '0001730168'

### Functions for downloading reports and making URLs

In [4]:
def download_reports(ticker, report_type, num_of_reports):
    dl = Downloader('./')
    dl.get(report_type, ticker, num_of_reports)
    
    path = f'./sec_edgar_filings/{ticker}/{report_type}'
    list_of_files = os.walk(path)
    list_of_files = list(list_of_files)[0][2]
    shutil.rmtree(path)
    
    return list_of_files

In [6]:
def make_json_urls(list_of_files, CIK):
    base_url = r"https://www.sec.gov/Archives/edgar/data/"
    hrefs = []
    
    for file in list_of_files:
        hrefs.append(base_url + CIK + '/' + file.replace('-','').replace('.htm','/index.json').replace('.txt','/index.json'))
    
    return hrefs

In [7]:
def make_xml_urls(json_urls):
    base_url = r"https://www.sec.gov"
    
    new_urls = []

    for url in json_urls:
        content = requests.get(url).json()

        for file in content['directory']['item']:
            if file['name'] == 'FilingSummary.xml':
                xml_summary = base_url + content['directory']['name'] + "/" + file['name']
                new_urls.append(xml_summary)
    
    return new_urls

In [8]:
def make_table_urls(xml_summary):
    base_url = xml_summary.replace('FilingSummary.xml', '')

    content = requests.get(xml_summary).content
    soup = BeautifulSoup(content, 'lxml')

    reports = soup.find('myreports')

    master_reports = []

    for report in reports.find_all('report')[:-1]:

        report_dict = {}
        report_dict['name_short'] = report.shortname.text
        report_dict['url'] = base_url + report.htmlfilename.text

        master_reports.append(report_dict)
        
    return master_reports

In [10]:
def make_statements_data(statements_url):
    
    statements_data = []

    for statement in statements_url:

        statement_data = {}
        statement_data['headers'] = []
        statement_data['sections'] = []
        statement_data['data'] = []

        content = requests.get(statement).content
        report_soup = BeautifulSoup(content, 'html')

        for index, row in enumerate(report_soup.table.find_all('tr')):

            cols = row.find_all('td')

            if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
                reg_row = [ele.text.strip() for ele in cols]
                statement_data['data'].append(reg_row)
            elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
                sec_row = cols[0].text.strip()
                statement_data['sections'].append(sec_row)
            elif (len(row.find_all('th')) != 0):            
                hed_row = [ele.text.strip() for ele in row.find_all('th')]
                statement_data['headers'].append(hed_row)
            else:            
                print('We encountered an error.')

        statements_data.append(statement_data)
        
    return statements_data

In [11]:
def make_df_from_report(i, statements_data):
    
    if len(statements_data[i]['headers']) > 1:
        income_header = statements_data[i]['headers'][1]
    else:
        income_header = statements_data[i]['headers'][0][1:]
    
    income_data = statements_data[i]['data']

    income_df = pd.DataFrame(income_data)

    income_df.index = income_df[0]
    income_df.index.name = 'Category'
    income_df = income_df.drop(0, axis = 1)

    income_df = income_df.replace('[\$,)]','', regex=True )\
                         .replace( '[(]','-', regex=True)\
                         .replace( '', 'NaN', regex=True)

    income_df = income_df.astype(float)
    income_df.columns = income_header
    
    return income_df

In [12]:
def make_final_data(all_master_reports):    
    datas = []
    statements_urls = []

    for i in range(len(all_master_reports)):
        for j in range(len(all_master_reports[i])):
            if 'R2.htm' in all_master_reports[i][j]['url'] or 'statements of operation' in all_master_reports[i][j]['name_short'].lower():
                statements_urls.append(all_master_reports[i][j]['url'])

    statements_data = make_statements_data(statements_urls)

    data = {}
    for j in range(len(statements_data)):
        key = all_master_reports[i][j]['name_short']
        data[key] = make_df_from_report(j, statements_data)
        df = data[key][:7]
        df = df.T.groupby(level=0).first().T
        datas.append(df)
        
    final_data = pd.concat(datas, axis=1)
    final_data = final_data.sort_index(axis=1)
    
    return final_data

In [13]:
report_type = '10-Q'
num_of_reports = 8

os.mkdir('./tables_from_reports')

for ticker, CIK in cik_dic.items():
    try:
        list_of_files = download_reports(ticker, report_type, num_of_reports)
        print('list_of_files отработала')
        json_urls = make_json_urls(list_of_files, CIK)
        print('json_urls отработала')
        xml_urls = make_xml_urls(json_urls)
        print('xml_urls отработала')
 
        all_master_reports = []
        for url in xml_urls:
            all_master_reports.append(make_table_urls(url))
        print('make_table_urls отработала')
        
        final_data = make_final_data(all_master_reports)
        final_data.to_excel(f'./tables_from_reports/{ticker}_last_{num_of_reports}_reports.xlsx')
        print(f'{ticker} - всё супер!')
    except:
        print(f'{ticker} - ОШИБКА!')

list_of_files отработала
json_urls отработала
xml_urls отработала
make_table_urls отработала
AAPL - ОШИБКА!
list_of_files отработала
json_urls отработала
xml_urls отработала
make_table_urls отработала
MSFT - всё супер!
list_of_files отработала
json_urls отработала
xml_urls отработала
make_table_urls отработала
AMZN - всё супер!
list_of_files отработала
json_urls отработала
xml_urls отработала
make_table_urls отработала
GOOG - всё супер!
list_of_files отработала
json_urls отработала
xml_urls отработала
make_table_urls отработала
GOOGL - всё супер!
list_of_files отработала
json_urls отработала
xml_urls отработала
make_table_urls отработала
TSLA - ОШИБКА!
list_of_files отработала
json_urls отработала
xml_urls отработала
make_table_urls отработала
FB - всё супер!
BABA - ОШИБКА!
list_of_files отработала
json_urls отработала
xml_urls отработала
make_table_urls отработала
BRK-B - ОШИБКА!
list_of_files отработала
json_urls отработала
xml_urls отработала
make_table_urls отработала
BRK-A - ОШИБК